In [1]:
import pickle
import spacy
from pathlib import Path

In [2]:
# import spacy

# def put_bos_eos(tokens_lists):
#     return [[("xxbos", "SPCL_BRIDGE"), *tokens, ("xxeos", "SPCL_BRIDGE")] for tokens in tokens_lists]

# def deal_with_caps(tokens_lists):
#     rlist = []
#     for tokens in tokens_lists:
#         nlist = []
#         for token in tokens:
#             nlist.append((token[0].lower(), token[1]))
#             if(token[0].isupper()):
#                 nlist.append(("xxcap", "SPCL_APPEND"))
#             elif(token[0][0].isupper()):
#                 nlist.append(("xxmaj", "SPCL_APPEND"))
#         rlist.append(nlist)
#     return rlist

# class Vocab():
#     def __init__(self, text_list, path_list=None, nlp=None, apply_defaults=True, callbacks=[]):
#         assert type(text_list) == list
#         assert type(callbacks) == list
#         self.nlp, self.apply_defaults = nlp, apply_defaults
#         self.callbacks = [put_bos_eos, deal_with_caps] if(self.apply_defaults) else []
#         self.callbacks.extend(callbacks)
#         if(self.nlp is None): self.nlp = spacy.load("en_core_web_sm")
#         self.tokens_lists = self.get_list_of_list_of_tokens(text_list)
#         self.tokens_lists = self.apply_callbacks(self.tokens_lists)
#         self.itos, self.stoi = self.create_vocab(self.tokens_lists)
#         self.batches = [[token for tokens in self.tokens_lists for token in tokens]]

#         self.tokens_base = Path('.')
#         self.dump_tokens(path_list, )
#         self.leaf_types = {
#             "ROOT_LEAF": 0,
#             "ONE_CHILD_LEAF": 1,
#             "TWO_CHILD_LEAF": 2,
#             "BRIDGE_LEAF": 3,
#             "INVARIANT": 4,
#         }
        
#         # NOTE: subject to change...INTJ is not tested
#         self.word_to_leaf_type = {
#             "PUNCT": self.leaf_types["BRIDGE_LEAF"],
#             "SYM": self.leaf_types["ONE_CHILD_LEAF"],
#             "ADJ": self.leaf_types["ONE_CHILD_LEAF"],
#             "ADP": self.leaf_types["ONE_CHILD_LEAF"],
#             "CCONJ": self.leaf_types["BRIDGE_LEAF"],
#             "NUM": self.leaf_types["ONE_CHILD_LEAF"],
#             "DET": self.leaf_types["ONE_CHILD_LEAF"],
#             "ADV": self.leaf_types["TWO_CHILD_LEAF"],
#             "X": self.leaf_types["ONE_CHILD_LEAF"],
#             "VERB": self.leaf_types["TWO_CHILD_LEAF"],
#             "NOUN": self.leaf_types["ONE_CHILD_LEAF"],
#             "PROPN": self.leaf_types["ONE_CHILD_LEAF"],
#             "PART": self.leaf_types["ONE_CHILD_LEAF"],
#             "INTJ": self.leaf_types["BRIDGE_LEAF"],
#             "PRON": self.leaf_types["ONE_CHILD_LEAF"],
#             "SPCL_APPEND": self.leaf_types["INVARIANT"],
#             "SPCL_BRIDGE": self.leaf_types["BRIDGE_LEAF"],
#             "ROOT": self.leaf_types["ROOT_LEAF"]
#         }

#     def __str__(self):
#         return f"{self.tokens_lists}\n\n{self.itos}\n\n{self.stoi}\n\n"
    
#     def __repr__(self):
#         return f"{self.tokens_lists}\n\n{self.itos}\n\n{self.stoi}\n\n"
    
#     def unsimplified_tagged_list(self, sentence):
#         tokens = word_tokenize(sentence)
#         return pos_tag(tokens)

#     def simplified_tagged_list(self, sentence):
#         doc = self.nlp(sentence)
#         tags = [(token.text, token.pos_) for token in doc]
#         return tags
    
#     def get_batched_token_sequences(self):
# #         self.batches =  [[token for tokens in self.tokens_lists for token in tokens]]
#         return self.batches
    
#     def get_list_of_list_of_tokens(self, text_list):
#         tokens_lists = []
#         for text in text_list:
#             tokens_lists.append(self.simplified_tagged_list(text))
#         return tokens_lists
    
#     def create_vocab(self, tokens_lists):
#         all_tokens = [item for sublist in tokens_lists for item in sublist]
#         all_tokens = set(all_tokens)
#         itos, stoi = [], {}
#         i = 0
#         for token, wtype in all_tokens:
#             stoi[token] = i
#             itos.append((token, wtype))
#             i += 1
#         return itos, stoi 
    
#     def apply_callbacks(self, tokens_lists):
#         for callback in self.callbacks:
#             tokens_lists = callback(tokens_lists)
#         return tokens_lists

#     def get_ltype(self, token):
#         return self.word_to_leaf_type[token]

#     def get_ltypes_dict(self):
#         return self.leaf_types
    
#     def get_seq_at_index(self, index):
#         return self.batches[index]
    
#     def get_indices_from_tokens(self, tokens):
#         return [self.stoi[token[0]] for token in tokens]

#     def get_word_tensors(self, tokens):
#         indices = self.get_indices_from_tokens(tokens)
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#         return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

In [18]:
class TextGenerator():
    def __init__(self, path_list, text_list=None, encoding="utf8"):
        assert type(path_list) == list or type(text_list) == list
        if(path_list is None):
            self.path_list = self.create_data_paths(text_list)
        self.nlp = spacy.load("en_core_web_sm")
        self.path_list, self.encoding = path_list, encoding
    
    def create_data_paths(self, base_path, text_list):
        i = 0
        paths = []
        for text in text_list:
            dest_path = base_path / f"{i}.txt"
            paths.append(dest_path)
            with open(dest_path, "w") as f:
                f.write(text)
        return paths
    
    def get_next_text(self):
        for p in self.path_list:
            with open(p, 'r',encoding=self.encoding) as f:
                s = f.read()
                yield p, s

In [19]:
class Vocab():
    def __init__(self, tokenizer, max_len=20000):
        self.tokenizer, self.max_len = tokenizer, max_len
        self.nlp = spacy.load("en_core_web_sm")
        self.itos, self.stoi, self.stoc = [], {}, {}
        self.special_tokens = ["xxbos", "xxeos", "xxmaj", "xxcap", "xxunk"]
        
    def trim_word_index_mappings(self):
        sorted_stoc = {k: v for k, v in reversed(sorted(self.stoc.items(), key=lambda item: item[1]))}
        for k in self.special_tokens:
            if k in sorted_stoc: del sorted_stoc[k]
        sorted_list = list(sorted_stoc.items())
        
        new_stoi, new_itos, new_stoc = {}, [], {}
        for spcl_tok in self.special_tokens:
            cur_ind = len(new_itos)
            new_stoi[spcl_tok] = cur_ind
            new_itos.append(spcl_tok)
            
        for i in range(min(self.max_len, len(sorted_list))):
            k, v = sorted_list[i]
            cur_ind = len(new_itos)
            new_stoc[k], new_stoi[k] = v, cur_ind
            new_itos.append(k)
        self.itos, self.stoi, self.stoc = new_itos, new_stoi, new_stoc
    
    def update_token_list(self, token_list):
        new_list = []
        for token in token_list:
            tok, ltype = token
            if tok not in self.stoi: new_list.append(("xxunk", "SPCL_APPEND"))
            else: new_list.append((tok, ltype))
        return new_list
    
    def push_tok_to_word_index_mappings(self, tok):
        if tok not in self.stoi:
            ind = len(self.itos)
            self.stoi[tok], self.stoc[tok] = ind, 1
            self.itos.append(tok)
        else:
            self.stoc[tok] += 1
        
    def prepare(self):
        for _,token_list in self.tokenizer.get_next_token_list():
            for tok,_ in token_list:
                self.push_tok_to_word_index_mappings(tok)
        self.trim_word_index_mappings()
        
    def get_trimmed_tokens(self, token_list):
        return self.update_token_list(token_list)    

    def get_indices_from_tokens(self, tokens):
        return [self.stoi[token[0]] for token in tokens]

    def get_word_tensors(self, tokens):
        indices = self.get_indices_from_tokens(tokens)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

In [30]:
class Tokenizer():
    def __init__(self, path_list, dest_path=Path('./data'), apply_defaults=True, max_len=20000, callbacks=[]):
        self.text_generator = TextGenerator(path_list)
        self.nlp = spacy.load("en_core_web_sm")
        self.tokens_paths = []
        self.callbacks = [self.put_bos_eos, self.deal_with_caps] if(apply_defaults) else []
        self.callbacks.extend(callbacks)
        self.prepare_initial_tokens(path_list, dest_path)
        self.vocab = Vocab(self, max_len)
        self.vocab.prepare()
        self.update_tokens_lists()
        
        self.leaf_types = {
            "ROOT_LEAF": 0,
            "ONE_CHILD_LEAF": 1,
            "TWO_CHILD_LEAF": 2,
            "BRIDGE_LEAF": 3,
            "INVARIANT": 4,
        }       
        self.word_to_leaf_type = {
            "PUNCT": self.leaf_types["BRIDGE_LEAF"],
            "SYM": self.leaf_types["ONE_CHILD_LEAF"],
            "ADJ": self.leaf_types["ONE_CHILD_LEAF"],
            "ADP": self.leaf_types["ONE_CHILD_LEAF"],
            "CCONJ": self.leaf_types["BRIDGE_LEAF"],
            "NUM": self.leaf_types["ONE_CHILD_LEAF"],
            "DET": self.leaf_types["ONE_CHILD_LEAF"],
            "ADV": self.leaf_types["TWO_CHILD_LEAF"],
            "X": self.leaf_types["ONE_CHILD_LEAF"],
            "VERB": self.leaf_types["TWO_CHILD_LEAF"],
            "NOUN": self.leaf_types["ONE_CHILD_LEAF"],
            "PROPN": self.leaf_types["ONE_CHILD_LEAF"],
            "PART": self.leaf_types["ONE_CHILD_LEAF"],
            "INTJ": self.leaf_types["BRIDGE_LEAF"],
            "PRON": self.leaf_types["ONE_CHILD_LEAF"],
            "SPCL_APPEND": self.leaf_types["INVARIANT"],
            "SPCL_BRIDGE": self.leaf_types["BRIDGE_LEAF"],
            "ROOT": self.leaf_types["ROOT_LEAF"]
        }
        
    def put_bos_eos(self, tokens_list):
        return [("xxbos", "SPCL_BRIDGE"), *tokens_list, ("xxeos", "SPCL_BRIDGE")]

    def deal_with_caps(self, token_list):
        nlist = []
        for token in token_list:
            nlist.append((token[0].lower(), token[1]))
            if(token[0].isupper()):
                nlist.append(("xxcap", "SPCL_APPEND"))
            elif(token[0][0].isupper()):
                nlist.append(("xxmaj", "SPCL_APPEND"))
        return nlist

    def apply_callbacks(self, token_list):
        for callback in self.callbacks:
            token_list = callback(token_list)
        return token_list

    def prepare_initial_tokens(self, src_paths, dest_root):
        for path, content in self.text_generator.get_next_text():
            fname = path.stem
            dest_path = dest_root / f"{fname}.pkl"
            self.tokens_paths.append(dest_path)
            token_list = self.simplified_tagged_list(content)
            token_list = self.apply_callbacks(token_list)
            with open(dest_path, 'wb') as f:
                pickle.dump(token_list, f)
    
    def get_next_token_list(self):
        for p in self.tokens_paths:
            with open(p, "rb") as f:
                tk_list = pickle.load(f)
            yield p, tk_list

    def update_tokens_lists(self):
        for p in self.tokens_paths:
            with open(p, "rb") as f:
                tk_list = pickle.load(f)
                new_tk_list = self.vocab.get_trimmed_tokens(tk_list)
            with open(p, "wb") as f:
                pickle.dump(new_tk_list, f)
    
    def simplified_tagged_list(self, sentence):
        doc = self.nlp(sentence)
        tags = [(token.text, token.pos_) for token in doc]
        return tags
    
    def get_ltype(self, token):
        return self.word_to_leaf_type[token]

    def get_ltypes_dict(self):
        return self.leaf_types
    


In [25]:
tk = Tokenizer([Path('./test.txt')], max_len=150)

In [52]:
p = Path('./test.txt')
p.stem

'test'

In [48]:
sentences = [
    'Hariom is playing',
    'I got milk from the store',
    'from the store, I got milk',
    'hey hey hey',
    'this sentence can be structured to a tree in a systematic manner',
    'what is he doing',
    'sentence illegal is',
    'sentence is legal',
    'This sentence is going to be long because we need to check what is going on',
    'The poop is kept on the beautiful table',
    'bread and butter is good for health',
    'wine is great with bread and butter is good for health'
]

v = Vocab(sentences)

In [49]:
v.tensor_from_tokens(v.tokens_lists[0])

tensor([[40],
        [44],
        [47],
        [21],
        [35],
        [ 2]])

In [31]:
def indexesFromSentence(vocab, sentence):
    return [vocab.stoi[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device="cpu").view(-1, 1)

In [32]:
import torch

In [33]:
tensorFromSentence(v, 'hariom is playing') # gives a column vector

tensor([[44],
        [21],
        [35]])

In [23]:
v.get_seq_at_index(0)

[('xxbos', 'SPCL_BRIDGE'),
 ('hariom', 'PROPN'),
 ('xxmaj', 'SPCL_APPEND'),
 ('is', 'VERB'),
 ('playing', 'VERB'),
 ('xxeos', 'SPCL_BRIDGE'),
 ('xxbos', 'SPCL_BRIDGE'),
 ('i', 'PRON'),
 ('xxcap', 'SPCL_APPEND'),
 ('got', 'VERB'),
 ('milk', 'NOUN'),
 ('from', 'ADP'),
 ('the', 'DET'),
 ('store', 'NOUN'),
 ('xxeos', 'SPCL_BRIDGE'),
 ('xxbos', 'SPCL_BRIDGE'),
 ('from', 'ADP'),
 ('the', 'DET'),
 ('store', 'NOUN'),
 (',', 'PUNCT'),
 ('i', 'PRON'),
 ('xxcap', 'SPCL_APPEND'),
 ('got', 'VERB'),
 ('milk', 'NOUN'),
 ('xxeos', 'SPCL_BRIDGE'),
 ('xxbos', 'SPCL_BRIDGE'),
 ('hey', 'INTJ'),
 ('hey', 'INTJ'),
 ('hey', 'INTJ'),
 ('xxeos', 'SPCL_BRIDGE'),
 ('xxbos', 'SPCL_BRIDGE'),
 ('this', 'DET'),
 ('sentence', 'NOUN'),
 ('can', 'VERB'),
 ('be', 'VERB'),
 ('structured', 'VERB'),
 ('to', 'ADP'),
 ('a', 'DET'),
 ('tree', 'NOUN'),
 ('in', 'ADP'),
 ('a', 'DET'),
 ('systematic', 'ADJ'),
 ('manner', 'NOUN'),
 ('xxeos', 'SPCL_BRIDGE'),
 ('xxbos', 'SPCL_BRIDGE'),
 ('what', 'NOUN'),
 ('is', 'VERB'),
 ('he', 'PR